In [1]:
import os
import sys


import tensorflow as tf
# sys.path.append("..")
# from cst_model import CSTModel
# import distortion_layers as ly
# import base_models as bm
# import callbacks as cb


sys.path.append("/main_dir/felipe/projects/processor_apps/pkgs/CST")
from cst.cst_model import CSTModel
from cst import distortion_layers as ly
from cst import base_models as bm
from cst import callbacks as cb


# tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR) 

In [2]:
os.getcwd()

'/main_dir/felipe/projects/CST/notebooks'

### Parameters


In [3]:
# train_path = "../data"
# train_path = "/main_dir/felipe/projects/cp_toolbox_data/artifacts_project/training_data/v4"
train_path = "/main_dir/felipe/projects/epi_seg/src/data/train/split/p16_S360_XR"

save_images = False  # utility class method to store the images and distorted images if True. 
                    # to make sure images are being properly created

tile_size = 128
batch_size = 32
channels = 3
# n_st_components = 2
alpha = 0
epochs = 10
class_mode = "binary"

validation_split = 0.2

### Create distortion layer

In [4]:
dcs = {
    "contrast": {"factor": [0.2, 0.2]},
    "color": {"factor": [20,0,20]},
    "blur": {"filter_shape": 2, "sigma": 5.},  # kernel size is 'filter_shape * 2 + 1'
    "brightness": {"lower": .85, "upper":1.15}
}

layers = [
    ly.RandomColorByChannel(**dcs["color"]), 
    tf.keras.layers.RandomContrast(**dcs["contrast"]),
    ly.RandomBrightness(**dcs["brightness"]),
    ly.RandomGaussianBlur(**dcs["blur"]),
    ly.BlueRedChannelSwapLayer(),
]


# dist_layer = [AdjSaturation(10),
#               tf.keras.layers.Lambda(lambda x: x * 0. + 55)]

dist_layer = layers



2023-08-01 16:05:24.632722: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 16:05:24.636455: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 16:05:24.636546: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-08-01 16:05:24.636967: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

### Load images and assign parameters

In [5]:
gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split = validation_split
)

t_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode='rgb',  # rgb for color
    batch_size=batch_size,
    class_mode=class_mode,  # 'sparse' for multiclass, 'binary' for binary 
    subset='training'
)

v_flow = gen.flow_from_directory(
    directory=train_path,
    target_size=(tile_size,tile_size),
    color_mode="rgb",  # rgb for color
    batch_size=batch_size,
    shuffle=False,
    class_mode=class_mode,  # 'sparse' for multiclass, 'binary' for binary
    subset='validation'
)

Found 41770 images belonging to 2 classes.
Found 10442 images belonging to 2 classes.


### Load and compile network

In [6]:
if class_mode == "binary":
    metrics = [
        tf.keras.metrics.BinaryCrossentropy(name="bce"),  # BinaryCrossentropy for binary
        tf.keras.metrics.BinaryAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.binary_crossentropy
    final_layer_node = 1
    binary = True
    
elif class_mode == "sparse":
    metrics = [
        tf.keras.metrics.SparseCategoricalCrossentropy(name="sce"), 
        tf.keras.metrics.SparseCategoricalAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.sparse_categorical_crossentropy
    final_layer_node = t_flow.num_classes
    binary = False

elif class_mode == "categorical":
    metrics = [
        tf.keras.metrics.CategoricalCrossentropy(name="cce"),
        tf.keras.metrics.CategoricalAccuracy(name="acc")
    ]
    base_loss = tf.keras.losses.categorical_crossentropy
    final_layer_node = t_flow.num_classes
    binary = False
    
else:
    print("no class mode provided")
    
    
base_model = bm.create_thesis_model(tile_size=tile_size, channels=3, final_layer_node=final_layer_node)
i = tf.keras.Input(shape=(tile_size, tile_size, channels))
x_i = base_model(i)
cst_model = CSTModel(inputs=i, outputs=x_i, alpha=alpha, 
                     dist_layers=dist_layer, binary=binary, save_images=save_images)


cst_model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3, amsgrad=True),
    loss = base_loss,
    metrics = metrics 
)


### Train network

In [7]:

cst_model.run_eagerly = True

cst_model.fit(
    x=t_flow,
    validation_data=v_flow,
    epochs=epochs,
    class_weight=bm.get_class_weights(t_flow.classes)
)

Epoch 1/10


2023-08-01 16:05:27.991396: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-01 16:05:28.307232: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100




tf.Tensor(0.6862759, shape=(), dtype=float32)
tf.Tensor(0.0012051621, shape=(), dtype=float32)
tf.Tensor(0.12270687, shape=(), dtype=float32)
   1/1306 [..............................] - ETA: 1:15:49 - loss_0: 0.6863 - cst_metric: 0.6863 - bce_: 0.6863 - acc_: 0.5625 - bce_0: 0.6862 - acc_0: 0.5938 - bce_1: 0.7080 - acc_1: 0.5625 - bce_2: 0.6608 - acc_2: 0.6875 - bce_3: 0.6821 - acc_3: 0.5000 - bce_4: 0.7468 - acc_4: 0.5000

tf.Tensor(1.3703686, shape=(), dtype=float32)
tf.Tensor(0.07268228, shape=(), dtype=float32)
tf.Tensor(0.10081835, shape=(), dtype=float32)
   2/1306 [..............................] - ETA: 11:49 - loss_0: 0.8573 - cst_metric: 0.7723 - bce_: 0.7718 - acc_: 0.5703 - bce_0: 0.7897 - acc_0: 0.5977 - bce_1: 0.7916 - acc_1: 0.5703 - bce_2: 0.7532 - acc_2: 0.6797 - bce_3: 0.7974 - acc_3: 0.5156 - bce_4: 0.8469 - acc_4: 0.5156  

tf.Tensor(1.6346352, shape=(), dtype=float32)
tf.Tensor(0.15120378, shape=(), dtype=float32)
tf.Tensor(0.32157925, shape=(), dtype=float32)
  

  20/1306 [..............................] - ETA: 12:05 - loss_0: 0.8588 - cst_metric: 0.9182 - bce_: 0.9195 - acc_: 0.5900 - bce_0: 0.9432 - acc_0: 0.6137 - bce_1: 0.9310 - acc_1: 0.5857 - bce_2: 0.9063 - acc_2: 0.6395 - bce_3: 0.9809 - acc_3: 0.5446 - bce_4: 0.9393 - acc_4: 0.5571

tf.Tensor(0.40442628, shape=(), dtype=float32)
tf.Tensor(0.27272242, shape=(), dtype=float32)
tf.Tensor(0.26045075, shape=(), dtype=float32)
  21/1306 [..............................] - ETA: 12:08 - loss_0: 0.8508 - cst_metric: 0.9148 - bce_: 0.9162 - acc_: 0.5916 - bce_0: 0.9392 - acc_0: 0.6148 - bce_1: 0.9275 - acc_1: 0.5873 - bce_2: 0.9038 - acc_2: 0.6401 - bce_3: 0.9769 - acc_3: 0.5467 - bce_4: 0.9346 - acc_4: 0.5586

tf.Tensor(0.29397756, shape=(), dtype=float32)
tf.Tensor(0.07124258, shape=(), dtype=float32)
tf.Tensor(0.1094065, shape=(), dtype=float32)
  22/1306 [..............................] - ETA: 12:08 - loss_0: 0.8427 - cst_metric: 0.9113 - bce_: 0.9129 - acc_: 0.5931 - bce_0: 0.9352 - acc_0: 

  39/1306 [..............................] - ETA: 11:57 - loss_0: 0.7407 - cst_metric: 0.8523 - bce_: 0.8560 - acc_: 0.6200 - bce_0: 0.8719 - acc_0: 0.6372 - bce_1: 0.8639 - acc_1: 0.6167 - bce_2: 0.8527 - acc_2: 0.6579 - bce_3: 0.9083 - acc_3: 0.5818 - bce_4: 0.8621 - acc_4: 0.5857

tf.Tensor(0.32691413, shape=(), dtype=float32)
tf.Tensor(0.1815521, shape=(), dtype=float32)
tf.Tensor(0.23720583, shape=(), dtype=float32)


KeyboardInterrupt: 